In [1]:
import json
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from constants import RUSSIAN_TIME_ZONES, rp5_columns
from interpolation import interpolate_mmx, create_patterns
from converters import convert_rp5_to_mmx, convert_raw_to_mmx, convert_mmx_to_mmcc_forecast, convert_raw_to_mmx
from loaders import load_rp5_stations, load_mm94_stations
from geographical import find_nearest_wmo_station

## Convert RP5 to Mmx

* Choose one mm94 station and finding the closeat wmo station

In [2]:
## find nearest wmo station
mm94_station_id = 20833

wmo_station_id = find_nearest_wmo_station(mm94_station_id)
print(wmo_station_id.min())

23412


* Load data from the nearest wmo station

In [3]:
rp5 = load_rp5_stations(wmo_station_id)

* Convert from RP5 to Mmx format

In [4]:
mmx_forecast = convert_rp5_to_mmx(rp5)
mmx_forecast_interpolated = interpolate_mmx(mmx_forecast, 20)
mmx_forecast_interpolated

,date_time_utc,data_precip_code,data_pressure,data_cloudiness,data_t_air,data_wind_velocity,data_dampness,data_dew_point,data_wind_speedmax,data_p_weather,data_wind_dir,data_precip_count,data_visibility,station_id,date_time
0,2011-12-31 21:00:00,0.0,759.600000,100.000000,-0.900000,2.000000,95.000000,-1.600000,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 00:00:00
1,2011-12-31 21:20:00,0.0,759.633333,100.000000,-0.911111,2.000000,95.000000,-1.611111,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 00:20:00
2,2011-12-31 21:40:00,0.0,759.666667,100.000000,-0.922222,2.000000,95.000000,-1.622222,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 00:40:00
3,2011-12-31 22:00:00,0.0,759.700000,100.000000,-0.933333,2.000000,95.000000,-1.633333,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 01:00:00
4,2011-12-31 22:20:00,0.0,759.733333,100.000000,-0.944444,2.000000,95.000000,-1.644444,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 01:20:00
5,2011-12-31 22:40:00,0.0,759.766667,100.000000,-0.955556,2.000000,95.000000,-1.655556,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 01:40:00
6,2011-12-31 23:00:00,0.0,759.800000,100.000000,-0.966667,2.000000,95.000000,-1.666667,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 02:00:00
7,2011-12-31 23:20:00,0.0,759.833333,100.000000,-0.977778,2.000000,95.000000,-1.677778,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 02:20:00
8,2011-12-31 23:40:00,0.0,759.866667,100.000000,-0.988889,2.000000,95.000000,-1.688889,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 02:40:00
9,2012-01-01 00:00:00,0.0,759.900000,100.000000,-1.000000,2.000000,95.000000,-1.700000,NaN,0.0,315.0,NaN,50000.000000,23412,2012-01-01 03:00:00


* Convert from Mmx to MmccForecast format

In [5]:
mmcc_forecast = convert_mmx_to_mmcc_forecast(mmx_forecast_interpolated)

## Convert Raw to Mmx

* Load Raw mm94 data

In [6]:
%%time
raw = load_mm94_stations(mm94_station_id)
raw.head()

CPU times: user 1.84 s, sys: 120 ms, total: 1.96 s
Wall time: 1.96 s


* Convert Raw to Mmx

In [7]:
mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis, 20)
mmx_rwis_interpolated = create_patterns(mmx_rwis_interpolated)

Column id_p_weather can not be calculated
Column data_visibility can not be calculated
Column id_visibility can not be calculated
Column data_p_weather can not be calculated


In [9]:
mmcc_rwis = convert_mmx_to_mmcc_forecast(mmx_rwis_interpolated)
mmcc_rwis

Column visibility can not be calculated
Column p_weather can not be calculated


,date_time_metro,wind_direction,wind_speed,precipitation_type,station_id,t_air,humidity,pressure,t_dew_point,precipitation_intensity,visibility,date_time_utc,p_weather,cloudiness
19,2008-03-23 21:00 UTC,9999.000000,9999.0,9999.0,1821,-2.733333,97.975000,9999.000000,-2.925000,9999.0,9999,2008-03-23 21:00:00,9999,9999.0
20,2008-03-23 21:20 UTC,9999.000000,9999.0,9999.0,1821,-2.666667,98.050000,9999.000000,-2.850000,9999.0,9999,2008-03-23 21:20:00,9999,9999.0
21,2008-03-23 21:40 UTC,9999.000000,9999.0,9999.0,1821,-2.600000,98.125000,9999.000000,-2.775000,9999.0,9999,2008-03-23 21:40:00,9999,9999.0
22,2008-03-23 22:00 UTC,9999.000000,9999.0,9999.0,1821,-2.533333,98.200000,9999.000000,-2.700000,9999.0,9999,2008-03-23 22:00:00,9999,9999.0
23,2008-03-23 22:20 UTC,9999.000000,9999.0,9999.0,1821,-2.466667,98.275000,9999.000000,-2.625000,9999.0,9999,2008-03-23 22:20:00,9999,9999.0
24,2008-03-23 22:40 UTC,9999.000000,9999.0,9999.0,1821,-2.400000,98.350000,9999.000000,-2.550000,9999.0,9999,2008-03-23 22:40:00,9999,9999.0
25,2008-03-23 23:00 UTC,9999.000000,9999.0,9999.0,1821,-2.333333,98.425000,9999.000000,-2.475000,9999.0,9999,2008-03-23 23:00:00,9999,9999.0
26,2008-03-23 23:20 UTC,9999.000000,9999.0,9999.0,1821,-2.266667,98.500000,9999.000000,-2.400000,9999.0,9999,2008-03-23 23:20:00,9999,9999.0
27,2008-03-23 23:40 UTC,9999.000000,9999.0,9999.0,1821,-2.200000,98.575000,9999.000000,-2.325000,9999.0,9999,2008-03-23 23:40:00,9999,9999.0
28,2008-03-24 00:40 UTC,9999.000000,9999.0,9999.0,1821,-2.050000,98.575000,9999.000000,-2.175000,9999.0,9999,2008-03-24 00:40:00,9999,9999.0
